# Final Capstone Project

### Business Problem:

   Using data on the venues present in different neighborhoods of Atlanta, GA combined with average rent of each neighborhood. We will train a linear regression model to predict rent prices of a neighborhood. This problem will require the use of Foursquare API to get local venues as well as data about rent and neighborhood location which will be aquired through various sources. This report should interest developers and landlords wondering where to build or how much to charge for rent as well as renters who are looking for a place to rent and are interested in how much is fair rent. Finally this report will also interest city officials who will be able to gain insight on certian venues effects on neighborhood rent.

### Backgroud:

   The neighborhoods that comprise the city of Atlanta are rich with history. Atlanta was the home of many civil rights leaders such as John Lewis, Ralph David Abernathy, Dr. Martin Luther King Jr, and many more. The neighborhoods they resided in are decorated with tributes to their historic neighbors. However the neighborhoods of Atlanta have also been shaped by its less glamourous history. During the American Civil War Atlanta was a crucial stronghold of the confederacy due to the multiple railroads that intersected within the city making the city crucial for sending supplies to confederate troops throughout the South East. In 1864 during the Union Army's "March to the Sea" led by General William Sherman the city of Atlanta was burnt down (with some help from the retreating confederates) and it's residents displaced. After the Civil War ended in 1865, Atlanta began its rebuilding process which was accelerated due to the cities extensive railroad network. This rebuilding led to a boom in the city's population and the city of Atlanta was made the capital city of Georgia soon after. The city experienced significant growth through the late 1800's and early 1900's however racial tensions and Jim Crow laws would make a lasting mark on the city. In 1906 the Atlanta Race Riots forced black residents out of the booming Downtown Atlanta into the Sweet Auburn neighborhood which remains a predominantly black neighborhood to this day. Further red lining and segregation led to some very noticable differences throughout the neighborhoods such as roads being renamed as they pass through historically "Black" or "White" neighborhoods (i.e. Boulevard becoming Monroe and Moreland becoming Braircliff). After sustained growth through the 20th century including the construction of the world's busiest airport in south Atlanta, several interstates that pass through the city, establishing of universities within the city such as Georgia Tech, Morehouse, Emory University. The huge growth of Atlanta throughout the 20th century was capstoned with the 1996 Summer Olympics being hosted in the city. Current day Atlanta is composed of an incredibly diverse, well-educated, and young population which has grown over 24% in the last decade making it one of the fastest growing cities in the U.S. 

### Data

   The Data that will be used to approach this problem will come from [RentCafe](https://www.rentcafe.com/average-rent-market-trends/us/ga/atlanta/) for the average rent per neighborhood, Geopy for the latitude and longitude data, and finally the Foursquare API to get venue data.  

#### Constructing Average Rent Dataframe

In [75]:
import pandas as pd
import numpy as np
from selenium import webdriver
from time import sleep
import requests

In [37]:
# creating lists
neighborhoods = []
rents = []

# initializing chrome driver and opening wikipedia page
browser = webdriver.Chrome()
browser.get('https://www.rentcafe.com/average-rent-market-trends/us/ga/atlanta/')
sleep(5)

# View all table cells
cookie = browser.find_element_by_xpath("//*[@id='cookieinfopolicy']/div")
cookie.click()
veiw_all = browser.find_element_by_xpath("//*[@id='MarketTrendsAverageRentTableFullView']")
veiw_all.click()

# locating table columns by xpath
neighbor = browser.find_elements_by_xpath("//*[@id='MarketTrendsAverageRentTable']//*[@class='current-row']/th")
rent = browser.find_elements_by_xpath("//*[@id='MarketTrendsAverageRentTable']//*[@class='current-row']/td")
print(len(neighbor))
print(len(rent))

# copying data of first column to neighborhoods list
for i in neighbor:
    neighborhoods.append(i.text)
    
# copying data of second column to rents list
for i in rent:
    rents.append(i.text)

browser.close()

# combining lists created above
lst = zip(neighborhoods, rents)

# creating dataframe containing list data
df = pd.DataFrame(lst, columns = ["Neighborhood", "Avg. Rent"])
df

632
632


,Neighborhood,Avg. Rent
0,Ansley Park,"$2,046"
1,Ardmore,"$2,046"
2,Brookwood Hills,"$2,046"
3,Collier Hills North,"$2,046"
4,Colonial,"$2,046"
...,...,...
627,Pomona Park,$950
628,Regency Point,$950
629,Southwest Atlanta,$950
630,Sunridge Construction,$950


#### Getting Latitude and Longitude of each neighborhood

In [38]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Atlanta_rent")

lat = []
long = []

# Use geopy to get Lat, Long for each neighborhood
for i in neighborhoods:
    try:
        location = geolocator.geocode(i + ", Atlanta, GA")
        lat.append(location.latitude)
        long.append(location.longitude)
    except AttributeError:
        # if no 
        lat.append(np.nan)
        long.append(np.nan)
        pass

lst = zip(neighborhoods, lat, long)

# creating dataframe containing list data
lat_long_atlanta = pd.DataFrame(lst, columns = ["Neighborhood", "Latitude", "Longitude"])
lat_long_atlanta

,Neighborhood,Latitude,Longitude
0,Ansley Park,33.794550,-84.376315
1,Ardmore,33.806282,-84.400028
2,Brookwood Hills,33.807883,-84.390205
3,Collier Hills North,NaN,NaN
4,Colonial,33.814009,-84.395760
...,...,...,...
627,Pomona Park,NaN,NaN
628,Regency Point,-7.745800,110.357041
629,Southwest Atlanta,45.004757,-84.151970
630,Sunridge Construction,NaN,NaN


#### Combine and Clean data

In [41]:
new_df = pd.merge(df, lat_long_atlanta, on="Neighborhood", how="inner")
new_df

,Neighborhood,Avg. Rent,Latitude,Longitude
0,Ansley Park,"$2,046",33.794550,-84.376315
1,Ardmore,"$2,046",33.806282,-84.400028
2,Brookwood Hills,"$2,046",33.807883,-84.390205
3,Collier Hills North,"$2,046",NaN,NaN
4,Colonial,"$2,046",33.814009,-84.395760
...,...,...,...,...
637,Pomona Park,$950,NaN,NaN
638,Regency Point,$950,-7.745800,110.357041
639,Southwest Atlanta,$950,45.004757,-84.151970
640,Sunridge Construction,$950,NaN,NaN


After combining the data sets I need to deal with missing values. With the code below I can see that Latitude and Longitude are missing 452 out of 642 values

In [44]:
len(new_df["Neighborhood"].unique())
missing_data = new_df.isnull()

for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")   

Neighborhood
False    642
Name: Neighborhood, dtype: int64

Avg. Rent
False    642
Name: Avg. Rent, dtype: int64

Latitude
True     452
False    190
Name: Latitude, dtype: int64

Longitude
True     452
False    190
Name: Longitude, dtype: int64



In [46]:
# drop rows with NaN in "Latitude" column
new_df.dropna(subset=["Latitude"], axis=0, inplace=True)

# drop rows with NaN in "Longitude" column
new_df.dropna(subset=["Longitude"], axis=0, inplace=True)

# reset index
new_df.reset_index(drop=True, inplace=True)
new_df

,Neighborhood,Avg. Rent,Latitude,Longitude
0,Ansley Park,"$2,046",33.794550,-84.376315
1,Ardmore,"$2,046",33.806282,-84.400028
2,Brookwood Hills,"$2,046",33.807883,-84.390205
3,Colonial,"$2,046",33.814009,-84.395760
4,Piedmont Park,"$2,046",33.789048,-84.371886
...,...,...,...,...
185,Cascade Heights,$950,33.722330,-84.463263
186,Cascade Road,$950,33.720639,-84.469215
187,Ivan Hill,$950,33.764858,-84.389215
188,Regency Point,$950,-7.745800,110.357041


Now that I have removed all the missing values, I am looking for incorrect location values by finding outliers in terms of distance from Atlanta

In [47]:
atlanta_lat = 33.753746
atlanta_long = -84.386330

# Calculating Distance from the center of Atlanta
new_df["Distance From ATL"] = ((new_df["Latitude"]-atlanta_lat)**2 + (new_df["Longitude"]-atlanta_long)**2)**0.5
new_df

,Neighborhood,Avg. Rent,Latitude,Longitude,Distance From ATL
0,Ansley Park,"$2,046",33.794550,-84.376315,0.042015
1,Ardmore,"$2,046",33.806282,-84.400028,0.054293
2,Brookwood Hills,"$2,046",33.807883,-84.390205,0.054275
3,Colonial,"$2,046",33.814009,-84.395760,0.060996
4,Piedmont Park,"$2,046",33.789048,-84.371886,0.038142
...,...,...,...,...,...
185,Cascade Heights,$950,33.722330,-84.463263,0.083100
186,Cascade Road,$950,33.720639,-84.469215,0.089252
187,Ivan Hill,$950,33.764858,-84.389215,0.011480
188,Regency Point,$950,-7.745800,110.357041,199.116029


In [52]:
new_df["Distance From ATL"].describe()

count    190.000000
mean       3.099603
std       17.372591
min        0.002349
25%        0.046797
50%        0.072657
75%        0.136748
max      199.116029
Name: Distance From ATL, dtype: float64

Removing outlier distances from data set. This will result in a data set in which every neighborhood is within a moderate distance of the center of Atlanta

In [82]:
# Defining an outlier as anything greater than 1.5 * 75th percentile
outlier = 0.136748 * 1.5

indexes = new_df[new_df["Distance From ATL"] > outlier].index
new_df.drop(indexes, inplace=True)
clean_df = new_df.drop_duplicates()
clean_df.reset_index(drop=True)

,Neighborhood,Avg. Rent,Latitude,Longitude,Distance From ATL
0,Ansley Park,"$2,046",33.794550,-84.376315,0.042015
1,Ardmore,"$2,046",33.806282,-84.400028,0.054293
2,Brookwood Hills,"$2,046",33.807883,-84.390205,0.054275
3,Colonial,"$2,046",33.814009,-84.395760,0.060996
4,Piedmont Park,"$2,046",33.789048,-84.371886,0.038142
...,...,...,...,...,...
155,Adams Park,$950,33.712052,-84.456873,0.081943
156,Campbellton Road,$950,33.693159,-84.517901,0.144851
157,Cascade Heights,$950,33.722330,-84.463263,0.083100
158,Cascade Road,$950,33.720639,-84.469215,0.089252


#### Find Venue Information per Neighborhood

In [71]:
# @hidden cell
CLIENT_ID = 'RD3GK2KL021H3ZNCPR1R0XA3FP3MIC1KYTKBF3NTNO43NL0X' # your Foursquare ID
CLIENT_SECRET = 'QD44DA4ZVUNOP3LXU0EBWBEMWMPZVGSWZAAVVZVWFAUDLJ3Q' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [73]:
def getNearbyVenues(names, latitudes, longitudes, rents, radius=500):
    
    venues_list=[]
    for name, lat, lng, rent in zip(names, latitudes, longitudes, rents):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng,
            rent,
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude',
                  'Neighborhood Rent',
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [83]:
# gets 100 nearby venues in each neighborhood
atlanta_venues = getNearbyVenues(names=new_df['Neighborhood'],
                                   latitudes= clean_df['Latitude'],
                                   longitudes=clean_df['Longitude'],
                                   rents=clean_df['Avg. Rent']
                                  )

Ansley Park
Ardmore
Brookwood Hills
Colonial
Piedmont Park
Lenox
Atlantic Station
Buckhead Forest
Buckhead Village
Peachtree Hills
Midtown Atlanta
Atlanta Memorial Park
Brookwood
Brookwood
Brookwood
Brookwood
Loring Heights
Atkins Park
Virginia Highland
Virginia Park Townhomes
Downtown Atlanta
Springlake
Virgilee Park
Candler Park
Inman Park
Lake Claire
Little Five Points
Lullwater
Oakland
Sweet Auburn
Castleberry Hill
Brandon
Castlewood
Margaret Mitchell
Paces
Bellwood Quarry
Bolton
Carey Park
Collier Heights
Collier Hills
Dupont Commons
Hills Park
Ridgewood Heights
Riverside
Riverside
Riverside
Riverside
Rockdale
Underwood Hills
Westover Plantation
Wildwood
Georgia Tech
Edgewood
Cabbagetown
Dunwoody Springs
Foxcroft
Glenview
Hammond Hills
Armour
Lindbergh
Reynoldstown
Bankhead
Center Hill
Grove Park
Stonewall
Woodland Hills
Woodland Hills
Woodland Hills
Woodland Hills
Benteen Park
Grant Park
Mechanicsville
Summerhill
East Atlanta
Ormewood Park
Chastain
Northland Ridge
The Glenridge
E

In [87]:
atlanta_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Neighborhood Rent,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Ansley Park,33.794550,-84.376315,"$2,046",Piedmont Atlanta BeltLine Sort-of-Urban Dare,33.792436,-84.373845,Trail
1,Ansley Park,33.794550,-84.376315,"$2,046",Cascades Garden,33.791002,-84.373586,Garden
2,Ansley Park,33.794550,-84.376315,"$2,046",Eclectic Bistro & Bar,33.793401,-84.372352,Bistro
3,Ansley Park,33.794550,-84.376315,"$2,046",Boy Next Door,33.794011,-84.371972,Men's Store
4,Ansley Park,33.794550,-84.376315,"$2,046",The Prado,33.792357,-84.373797,Miscellaneous Shop
...,...,...,...,...,...,...,...,...
2312,Providence Place,33.764858,-84.389215,$950,Caribou Coffee,33.762592,-84.385392,Coffee Shop
2313,Providence Place,33.764858,-84.389215,$950,Starbucks,33.761693,-84.385642,Coffee Shop
2314,Providence Place,33.764858,-84.389215,$950,Dunkin',33.762788,-84.385439,Donut Shop
2315,Providence Place,33.764858,-84.389215,$950,Folk Art Park At Ralph Mcgill & Courtland,33.764520,-84.384341,Sculpture Garden


In [94]:
# one hot encoding
atlanta_onehot = pd.get_dummies(atlanta_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood and rent columns back to dataframe
atlanta_onehot['Neighborhood'] = atlanta_venues['Neighborhood']
atlanta_onehot['Neighborhood Rent'] = atlanta_venues['Neighborhood Rent']

# moves Neighborhood and Neighborhood Rent to front of DF
cols_at_front = ['Neighborhood', 'Neighborhood Rent']
atlanta_onehot = atlanta_onehot[[c for c in cols_at_front if c in atlanta_onehot] + [c for c in atlanta_onehot if c not in cols_at_front]]

atlanta_onehot.tail()

,Neighborhood,Neighborhood Rent,ATM,Accessories Store,Adult Education Center,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Art Gallery,...,Warehouse Store,Waste Facility,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
2312,Providence Place,$950,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2313,Providence Place,$950,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2314,Providence Place,$950,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2315,Providence Place,$950,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2316,Providence Place,$950,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [95]:
# gets frequency of venue per neighborhood as an average value of venue type
atlanta_grouped = atlanta_onehot.groupby(['Neighborhood', 'Neighborhood Rent']).mean().reset_index()
atlanta_grouped

,Neighborhood,Neighborhood Rent,ATM,Accessories Store,Adult Education Center,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Art Gallery,...,Warehouse Store,Waste Facility,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Adair Park,"$1,002",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Adamsville,"$1,116",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Amhurst,"$1,002",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Ansley Park,"$2,046",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Ardmore,"$2,046",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,Westview Cemetery,"$1,002",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
136,Wildwood,"$1,541",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
137,Wildwood East,"$1,040",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
138,Woodland Hills,"$1,447",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [105]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[2:]
    row_categories_sorted = row_categories.sort_values(axis=0,ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood', 'Neighborhood Rent']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = atlanta_grouped['Neighborhood']
neighborhoods_venues_sorted['Neighborhood Rent'] = atlanta_grouped['Neighborhood Rent']

for ind in np.arange(atlanta_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 2:] = return_most_common_venues(atlanta_grouped.iloc[ind, :], num_top_venues)

# resulting data frame contains each neighborhood and their 10 most common venue types
neighborhoods_venues_sorted.head()

,Neighborhood,Neighborhood Rent,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adair Park,"$1,002",General Travel,Gaming Cafe,Zoo Exhibit,Field,Financial or Legal Service,Fish & Chips Shop,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop
1,Adamsville,"$1,116",Park,Pool,General Entertainment,Plaza,Playground,Flower Shop,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service
2,Amhurst,"$1,002",Grocery Store,Mexican Restaurant,Park,Shipping Store,Gym / Fitness Center,Pet Store,Sandwich Place,Pharmacy,Breakfast Spot,Fondue Restaurant
3,Ansley Park,"$2,046",Bistro,Men's Store,Garden,Miscellaneous Shop,Flower Shop,Trail,Field,Financial or Legal Service,Fish & Chips Shop,Fondue Restaurant
4,Ardmore,"$2,046",Gym,Park,Pool,Food Truck,Photography Studio,Trail,Food Court,Food & Drink Shop,Food,Fondue Restaurant
